In [1]:
import pandas as pd
import requests

from tqdm import tqdm

import time
from random import random

In [2]:
links_path = 'dataset/ml-latest-small/links_with_wikidata_uri.csv'
dtype = {"movieId": str, "imdbId": str, "tmdbId": str, "wikidataURI": str}
links_df = pd.read_csv(links_path, dtype=dtype)

movie_path = 'dataset/ml-latest-small/movies.csv'
movies_df = pd.read_csv(movie_path, index_col='movieId')

Get movie value relations

In [ ]:
def get_value_relations(wikidata_uri: str) -> dict[str, str]:
    url = 'https://query.wikidata.org/sparql'
    query = f'''
    SELECT ?title ?year ?duration ?rottenTomatoesTommatometerScore ?boxOfficeWorldwide WHERE {{
    BIND(wd:{wikidata_uri} AS ?movie)
    OPTIONAL {{ ?movie wdt:P1476 ?title. }}
    OPTIONAL {{
        ?movie p:P577 ?publicationDateStatement.
        ?publicationDateStatement ps:P577 ?publicationDate;
                                pq:P291 ?country.
        BIND(YEAR(?publicationDate) as ?year)
        }}
    OPTIONAL {{ ?movie wdt:P2047 ?duration. }}
    OPTIONAL {{
        ?movie p:P444 ?reviewScoreStatement.
        ?reviewScoreStatement ps:P444 ?rottenTomatoesTommatometerScore;
                                pq:P459 wd:Q108403393;
                                pq:P447 wd:Q105584.
        }}
    OPTIONAL {{
        ?movie p:P2142 ?boxOfficeStatement.
        ?boxOfficeStatement ps:P2142 ?boxOfficeWorldwide;
                                pq:P3005 wd:Q13780930.
        }}
    }}
    '''
    r = requests.get(url, params = {'format': 'json', 'query': query})
    if r.status_code != 200: # if too many requests
        time.sleep(5*random())
        return get_value_relations(wikidata_uri)
    data = r.json()
    results = data['results']['bindings'][0]
    for key in results.keys():
        results[key] = results[key]['value']
    return results

In [ ]:
movies_df['year'] = None
movies_df['duration'] = None
movies_df['boxOfficeWorldwide'] = None
movies_df['tommatometerScore'] = None

Saving the value relations

In [ ]:
for _, row in tqdm(list(links_df.iterrows())):
    time.sleep(random()/3)
    value_relations = get_value_relations(row['wikidataURI'])
    if 'rottenTomatoesTommatometerScore' in value_relations:
        movies_df.loc[int(row['movieId']), 'tommatometerScore'] = value_relations['rottenTomatoesTommatometerScore']
    if 'year' in value_relations:
        movies_df.loc[int(row['movieId']), 'year'] = value_relations['year']
    if 'duration' in value_relations:
        movies_df.loc[int(row['movieId']), 'duration'] = value_relations['duration']
    if 'boxOfficeWorldwide' in value_relations:
        movies_df.loc[int(row['movieId']), 'boxOfficeWorldwide'] = value_relations['boxOfficeWorldwide']

In [ ]:
movies_df.head()

In [ ]:
movies_df.reset_index()
movies_df.to_csv('dataset/ml-latest-small/movies_with_wikidata_values.csv')

Get cast member relations

In [ ]:
def get_cast_members(wikidata_uri: str) -> list[dict[str, str]]:
    url = 'https://query.wikidata.org/sparql'
    query = f'''
    SELECT ?castMember ?castMemberLabel WHERE {{
    BIND(wd:{wikidata_uri} AS ?movie)
    OPTIONAL {{ ?movie wdt:P161 ?castMember. }}
    OPTIONAL {{ ?movie wdt:P725 ?castMember. }}
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    '''
    r = requests.get(url, params = {'format': 'json', 'query': query})
    if r.status_code != 200: # if too many requests
        time.sleep(5*random())
        return get_cast_members(wikidata_uri)
    data = r.json()
    results = data['results']['bindings']
    return [
        {
            'wikidata_uri': r['castMember']['value'][31:],
            'name': r['castMemberLabel']['value'] if 'castMemberLabel' in r else ''
        }
        for r in results if 'castMember' in r
    ]

In [ ]:
actors = {}
stars = []

In [ ]:
for _, row in tqdm(list(links_df.iterrows())[1210:]):
    time.sleep(random()/3)
    for cast_member in get_cast_members(row['wikidataURI']):
        actors[cast_member['wikidata_uri']] = cast_member['name']
        stars.append({'movieId': int(row['movieId']), 'castMember': cast_member['wikidata_uri']})

actors_df = pd.DataFrame(list(actors.items()), columns=['wikidataURI', 'name'])
stars_df = pd.DataFrame(stars)

In [ ]:
actors_df.head()

In [ ]:
stars_df.head()

In [ ]:
actors_df.to_csv('dataset/ml-latest-small/actors.csv')
stars_df.to_csv('dataset/ml-latest-small/stars.csv')

Get other relations

In [3]:
def get_relation(wikidata_uri: str, relation_uri: str, relation_name: str) -> list[dict[str, str]]:
    url = 'https://query.wikidata.org/sparql'
    query = f'''
    SELECT ?{relation_name} ?{relation_name}Label WHERE {{
    BIND(wd:{wikidata_uri} AS ?movie)
    OPTIONAL {{ ?movie wdt:{relation_uri} ?{relation_name}. }}
    SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    '''
    r = requests.get(url, params = {'format': 'json', 'query': query})
    if r.status_code != 200: # if too many requests
        time.sleep(5*random())
        return get_relation(wikidata_uri, relation_uri, relation_name)
    data = r.json()
    results = data['results']['bindings']
    return [
        {
            'wikidata_uri': r[relation_name]['value'][31:],
            'name': r[relation_name+'Label']['value'] if relation_name+'Label' in r else ''
        }
        for r in results if relation_name in r
    ]

In [ ]:
relation_uris = {
    'country': 'P495',
    'language': 'P364',
    'color': 'P462',
    'director': 'P57',
    'producer': 'P162',
    'execProducer': 'P1431',
    'composer': 'P86',
    'company': 'P272',
    'screenWriter': 'P58',
    'influencedBy': 'P737',
    'followedBy': 'P156',
    'follows': 'P155',
    'genre': 'P136',
    'creator': 'P170'
}

for relation_name, relation_uri in relation_uris.items():
    print(get_relation(links_df['wikidataURI'][4332], relation_uri, relation_name))

Get directors

In [ ]:
directors = {}
directed_by = []

for _, row in tqdm(list(links_df.iterrows())):
    time.sleep(random()/3)

    for director in get_relation(row['wikidataURI'], 'P57', 'director'):
        directors[director['wikidata_uri']] = director['name']
        directed_by.append({'movieId': int(row['movieId']), 'director': director['wikidata_uri']})

directors_df = pd.DataFrame(list(directors.items()), columns=['wikidataURI', 'name'])
directed_by_df = pd.DataFrame(directed_by)

In [ ]:
directors_df.to_csv('dataset/ml-latest-small/directors.csv')
directed_by_df.to_csv('dataset/ml-latest-small/directed_by.csv')

Get company

In [4]:
companies = {}
produced_by = []

In [8]:
for _, row in tqdm(list(links_df.iterrows())[900:]):
    time.sleep(random()/3)

    for company in get_relation(row['wikidataURI'], 'P272', 'company'):
        companies[company['wikidata_uri']] = company['name']
        produced_by.append({'movieId': int(row['movieId']), 'company': company['wikidata_uri']})

companies_df = pd.DataFrame(list(companies.items()), columns=['wikidataURI', 'name'])
produced_by_df = pd.DataFrame(produced_by)

In [11]:
companies_df.to_csv('dataset/ml-latest-small/companies.csv')
produced_by_df.to_csv('dataset/ml-latest-small/produced_by.csv')

Get country

In [12]:
countries = {}
produced_in = []

In [13]:
for _, row in tqdm(list(links_df.iterrows())):
    time.sleep(random()/3)

    for country in get_relation(row['wikidataURI'], 'P495', 'country'):
        countries[country['wikidata_uri']] = country['name']
        produced_in.append({'movieId': int(row['movieId']), 'country': country['wikidata_uri']})

countries_df = pd.DataFrame(list(countries.items()), columns=['wikidataURI', 'name'])
produced_in_df = pd.DataFrame(produced_in)

100%|██████████| 9742/9742 [4:14:41<00:00,  1.57s/it]  


In [14]:
countries_df.to_csv('dataset/ml-latest-small/countries.csv')
produced_in_df.to_csv('dataset/ml-latest-small/produced_in.csv')